In [ ]:
# Author: Arthur BARREAU
# Date: 2025-07-15
# R version: 4.3.3
# Description: This script generates a plot of the Antarctic sea ice extent
# using daily data (>=15% sea ice concentration). It includes historical 
# quantiles (1981–2010) and selected years for comparison. 
# Sources: site internet : https://nsidc.org/arcticseaicenews/charctic-interactive-sea-ice-graph/) 
#          data          : https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/data/
# ==============================================================================

# ==============================================================================
# 0. LOAD LIBRARIES ----
# ==============================================================================

library(ggplot2)
library(lubridate)
library(dplyr)
library(jsonlite)

options(timeout = 3600)

# ==============================================================================
# 1. PARAMETERS AND INPUT LOADING ----
# ==============================================================================

json_data <- fromJSON("galaxy_inputs/galaxy_inputs.json")
color             <- json_data$color
year              <- json_data$year
median            <- json_data$median
width_input       <- json_data$pngWidth
height_input      <- json_data$pngHeight
resolution_input  <- json_data$pngResolution
year_list         <- strsplit(year, ",")[[1]]

# Colors fallback if not provided ---
if (!is.null(color)) {
  list_color <- strsplit(color, ",")[[1]]
  color_list <- gsub("__pd__", "#", list_color)
}else{color_list <- NULL}

if(!is.null(json_data$plotXsettings)){
    if(json_data$plotXsettings == "pdf" || json_data$plotXsettings == "PDF"){
      pdf <- TRUE
    } else {
      pdf <- FALSE
      list_setting <- trimws(unlist(strsplit(json_data$plotXsettings, ",")))
      dpi_img <- as.numeric(list_setting[1])
      width_img <- as.numeric(list_setting[2])
      height_img <- as.numeric(list_setting[3])
    }

}
pdf_mode <- if (exists("pdf_mode")) pdf_mode else FALSE
dpi_img    <- if (exists("dpi_img")) dpi_img else NULL
width_img  <- if (exists("width_img")) width_img else NULL
height_img <- if (exists("height_img")) height_img else NULL


# ==============================================================================
# 2. FUNCTIONS ----
# ==============================================================================

load_data <- function(file_path) {
  data <- read.csv(file_path, header = TRUE)
  data <- data[-1, ]
  data$Extent <- as.numeric(data$Extent)
  data$Date <- as.Date(with(data, paste(Year, Month, Day, sep = "-")), "%Y-%m-%d")
  data$DayOfYear <- yday(data$Date)
  return(data)
}

filter_data_by_years <- function(data, selected_years) {
  return(data %>% filter(Year %in% selected_years))
}

calculate_quantiles <- function(data) {
  base <- data %>% filter(Year >= 1981 & Year <= 2010)
  return(base %>%
    group_by(DayOfYear) %>%
    summarise(
      MedianExtent = median(Extent, na.rm = TRUE),
      Q1 = quantile(Extent, 0.25, na.rm = TRUE),
      Q3 = quantile(Extent, 0.75, na.rm = TRUE),
      D10 = quantile(Extent, 0.10, na.rm = TRUE),
      D90 = quantile(Extent, 0.90, na.rm = TRUE)
    ) %>% mutate(Year = "1981-2010"))
}

apply_spline_interpolation <- function(df) {
  data.frame(
    DayOfYear = seq(min(df$DayOfYear), max(df$DayOfYear), length.out = 100),
    Extent = spline(df$DayOfYear, df$MedianExtent, n = 100)$y,
    Q1 = spline(df$DayOfYear, df$Q1, n = 100)$y,
    Q3 = spline(df$DayOfYear, df$Q3, n = 100)$y,
    D10 = spline(df$DayOfYear, df$D10, n = 100)$y,
    D90 = spline(df$DayOfYear, df$D90, n = 100)$y
  )
}

apply_spline_on_selected_data <- function(df) {
  df %>%
    group_by(Year) %>%
    group_modify(~ {
      out <- as.data.frame(spline(.x$DayOfYear, .x$Extent, n = 100))
      names(out) <- c("DayOfYear", "Extent")
      out$Year <- unique(.x$Year)
      return(out)
    })
}

create_base_plot <- function(data, day_breaks, month_labels, colors) {
  ggplot(data, aes(x = DayOfYear, y = Extent)) +
    scale_x_continuous(breaks = day_breaks, labels = month_labels, expand = c(0, 0)) +
    scale_y_continuous(limits = c(0, 22.5), breaks = seq(2.5, 22.5, 2.5), expand = c(0, 0)) +
    scale_linetype_manual(NULL, values = c("1981-2010 Median" = "solid")) +
    scale_fill_manual(NULL, values = c("Interquartile Range" = "gray25", "Interdecile Range" = "gray60")) +
    scale_color_manual(NULL, values = colors) +
    labs(title = "Antarctic Sea Ice Extent",
         subtitle = "Ocean area with at least 15% sea ice (in million km²)",
         x = "Date", y = "Extent (Million km²)") +
    theme_bw() +
    theme(
      plot.title = element_text(hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(hjust = 0.5, face = "italic")
    )
}

add_quantiles_to_plot <- function(p, quantiles) {
  p +
    geom_line(data = quantiles, aes(x = DayOfYear, y = Extent, linetype = "1981-2010 Median"), color = "black", linewidth = 0.7) +
    geom_ribbon(data = quantiles, aes(x = DayOfYear, ymin = Q1, ymax = Q3, fill = "Interquartile Range"), alpha = 0.3) +
    geom_ribbon(data = quantiles, aes(x = DayOfYear, ymin = D10, ymax = D90, fill = "Interdecile Range"), alpha = 0.3)
}

add_year_lines_to_plot <- function(p, year_data) {
  p + geom_line(data = year_data, aes(color = as.factor(Year)), linewidth = 0.7)
}

# ==============================================================================
# 3. PROCESSING ----
# ==============================================================================

download.file("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/data/S_seaice_extent_daily_v3.0.csv", "S_seaice_extent_daily_v3.0.csv")
data <- load_data("S_seaice_extent_daily_v3.0.csv")

# Fallback color palette
if (is.null(color_list) || length(color_list) < length(year_list)) {
  if (length(year_list) <= 8) {
    color_list <- rainbow(length(year_list))
  } else {
    color_list <- grDevices::colorRampPalette(RColorBrewer::brewer.pal(9, "Set1"))(length(year_list))
  }
}
colors_named <- setNames(color_list, year_list)

filtered <- filter_data_by_years(data, year_list)
quantiles <- calculate_quantiles(data)
spline_quantiles <- apply_spline_interpolation(quantiles)
spline_years <- apply_spline_on_selected_data(filtered)

# ==============================================================================
# 4. PLOTTING ----
# ==============================================================================

days <- c(1,32,61,92,122,153,183,214,245,275,306,336,365)
months <- c("01_Jan","01_Feb","01_Mar","01_Apr","01_May","01_Jun","01_Jul","01_Aug","01_Sep","01_Oct","01_Nov","01_Dec","31_Dec")

plot <- create_base_plot(spline_years, days, months, colors_named)
if (median) {
  plot <- add_quantiles_to_plot(plot, spline_quantiles)
}
plot <- add_year_lines_to_plot(plot, spline_years)
if(pdf==TRUE){
    pdf("outputs/Fig2.pdf")
    print(plot)
    dev.off()
} else {
  args <- list(filename = "outputs/Fig2.png", plot = plot)

  if (exists("dpi_img") && !is.null(dpi_img)) {
    args$dpi <- dpi_img
  }
  if (exists("width_img") && !is.null(width_img)) {
    args$width <- width_img
    args$units <- "px"
  }
  if (exists("height_img") && !is.null(height_img)) {
    args$height <- height_img
    args$units <- "px"
  }

  # Tenter de sauvegarder avec ggsave ou fonction compatible
  try(do.call(ggsave, args), silent = TRUE)

  # Fallback si le fichier n'a pas été créé
  if (!file.exists("outputs/Fig2.png")) {
    png("outputs/Fig2.png")
    print(plot)
    dev.off()
  }
}
